# Sparkify Data Lakes with Spark (on AWS EMR)
This notebook creates a data lake from data hosted in S3 and executes the necessary ETL instructions using Spark in an AWS EMR cluster.

In [1]:
import sys, os
from pathlib import Path
from pyspark.sql import DataFrame
from IPython.core import display as ICD

In [2]:
src_path: str = "../src"
sys.path.append(src_path)

In [3]:
from etl import main as run_etl
from utils import process_config, create_spark_session

In [4]:
user_config, dl_config = (
    process_config(Path(os.getcwd()).parent.joinpath("_user.cfg")),
    process_config(Path(os.getcwd()).parent.joinpath("dl.cfg")),
)
spark = create_spark_session(user_config, dl_config)

22/11/10 16:33:35 WARN Utils: Your hostname, uzi resolves to a loopback address: 127.0.1.1; using 192.168.1.181 instead (on interface wlp114s0)
22/11/10 16:33:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/uziel/miniconda3/envs/sparkify_spark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/uziel/.ivy2/cache
The jars for the packages stored in: /home/uziel/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-091157ed-dd26-4363-bd16-dc464f11853c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 151ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-----------------------------------

22/11/10 16:33:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. Extract, Transform and Load data (ETL)

In [5]:
run_etl()

INFO:root:Bucket spark-dl already exists.
INFO:root:Available buckets: [s3.Bucket(name='spark-dl'), s3.Bucket(name='uziel-at-udacity')]
INFO:root:Loading song data...


22/11/10 16:33:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/11/10 16:33:43 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
22/11/10 16:33:43 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
22/11/10 16:33:43 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.


INFO:root:Song data successfully loaded.                                        
INFO:root:Creating songs table...


## 2. Perform example queries

### 2.1. Get first 5 rows of every table

In [ ]:
bucket_prefix = f"s3a://{dl_config.get('S3', 'DEST_BUCKET_NAME')}"
tables = ("songs", "artists", "users", "time", "songplays")

In [ ]:
for table in tables:
    print(f"First 5 rows of {table}:")
    table_df = spark.read.parquet(f"{bucket_prefix}/{table}")
    ICD.display(DataFrame(table_df.take(5)).toPandas())
    print(f"The full table contains a total of {table_df.count()} records")

### 2.2. Who are the top 5 users with the highest activity?

In [ ]:
songplays_df = spark.read.parquet(f"{bucket_prefix}/songplays")

In [ ]:
ICD.display(
    DataFrame(
        songplays_df.groupBy("user_id").count().sort("count", ascending=False)
    ).toPandas()
)